# Bu yarışmada **Kanun Hükmünde Kararname** dosyaları için 91 belgenin incelendikten sonra bizden beklenilen görevler:

- "kategori" alanının tespiti. Bu görevle ilgili olan [Mevcut Çalışmamız](https://github.com/iflGARAJI10100/taMAM/blob/main/aramaProjesiKategoriBulmaKHK%20(1).ipynb) için tıklayınız.
- "mukerrer_no" alanının tespiti. Bu görevle ilgili olan [Mevcut Çalışmamız](https://github.com/iflGARAJI10100/taMAM/blob/main/aramaProjesiVeriIslemeMukerrerNoTespitModulu.ipynb) için tıklayınız.
- "madde_sayisi" alanının tespiti. Bu görevle ilgili olan [Mevcut Çalışmamız](https://github.com/iflGARAJI10100/taMAM/blob/main/aramaProjesiMaddeSayisiBulmaMod%C3%BCl%C3%BC.ipynb) için tıklayınız.
- "rega_no, rega_tarihi, mevzuat_no ve mevzuat_tarihi" alanlarının tespiti. Bu kodda ise bu alanalın tespitiyle alakalı çalışmalarımız mevcuttur.

Veri işleme ve analizi için pandas kütüphanesi kullanılacaktır. Pandas kütüphanesi pd rumuzu ile kısaltılmıştır.

Tarih verilerinin işlenmesi için **datetime** kütüphanesi kullanılacaktır.

In [25]:
import pandas as pd
from datetime import datetime

Verilerimiz Google Drive (bulut) üzerinde olduğu için Google Colab ile Google Drive arasında bağlantı kurulacaktır.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Google Drive içinde Colab Notebooks klasörünün içinde TDDI2022 isimli klasörde verilerimizi sakladığımız için bu klasöre erişilecektir.

***NOT:*** Bu klasörü daha önce oluşturmadıysanız %md TDDI2022 komutu veya Google Drive Ara Yüzü ile klasörü oluşturunuz!

In [ ]:
%cd drive/
%cd MyDrive/
%cd Colab Notebooks/
%cd TDDI2022/

# Klasörün içeriğinin görülmesi için
%ls

Veri İskeleti TDDI2022 klasöründe bulunan yarışma kapsamında yarışmacılara sağlanan **kanunum-nlp-doc-analysis-dataset.csv** dosyasının içindeki verileri pandas ile okuyarak oluşturulacaktır.

Veri iskeleti oluşturulduktan sonra veri iskeleti ile ilgili bilgi ekrana yazdırılacaktır. Tam özetin yazdırılıp yazdırılmayacağı durumunu *verbose=True* (yazdırılsın) parametresi ile kontrol edilecektir. 

In [ ]:
df = pd.read_csv('kanunum-nlp-doc-analysis-dataset.csv')
df.info(verbose=True)

 Veri İskeletinde **kategori** sütununun / alanının kaç gruptan ve bu grupların kaç satırdan / kayıttan oluştuğu ekrana yazdırılacaktır.

In [ ]:
print(df.groupby('kategori').size())

Verilerde yapacağımız ön çalışmada bizden istenilen değerleri araştıracağımız **Kanun Hükmünde Kararname** belgelerinin özelliklerini inceliyoruz.

In [ ]:
print(df.loc[df['kategori'] == 'Kanun Hükmünde Kararname'])

Verilerde yaptığımız ek çalışmalarda ise veri kümesinin data_text sütunundaki / alanındaki içerik iki parçaya ayrılabileceği görülmüştür. Böylelikle data_text sütunu / alanı işlenirken belgeye ait bilgiler birinci parçada belgenin içeriği ikinci parçada görülebilir. Tüm data_text kayıtlarında yer alan ortak bir noktanın varlığını kontrol etmek için aşağıdaki kod yardımı ile "Resmi belgelerin hepsinde 'Madde 1' verisi var mı?" önermesini kontrol ediyoruz.

In [ ]:
kayitSayisi = 91say = 0
satirToplami, sutunToplami = df.shape
for satir in range(satirToplami):
  if df['data_text'][satir].find('Madde 1 -') != -1 and df['kategori'][satir] == 'Kanun Hükmünde Kararname':
    say+=1
if say==kayitSayisi:
  print(say,'=',kayitSayisi,'yani data_text alanındaki toplam KHK belgesi sayısı ve her bir kayıtta "Madde 1" verisini içeren KHK belgelerinin sayısı birbirine eşittir.')
else:
  print('Tüm KHK belgelerinde aranan veri bulunamamıştır.')

Yukarıdaki bulguların önermemizi kanıtlaması doğrultusunda örnek bir kaydı ikiye bölerek **belge ile ilgili verileri toplayabileceğimiz baş kısmı** görelim. Yine bu baş kısmın **bizden istenilen değerlerler** ile ilgili bilgi içerip içermediğini görmek için dosyada mevcut olan veriyi de ekrana yazdıralım.

In [ ]:
satir = 826
print('\nKategori: ',df['kategori'][satir],'\n','Resmi Gazete No: ',df['rega_no'][satir],'\n','Resmi Gazete Tarihi: ',df['rega_tarihi'][satir],'\n','Mevzuat No: ',df['mevzuat_no'][satir],'\n','Mevzuat Tarihi: ',df['mevzuat_tarihi'][satir],'\n',35*'-','\nİçerik:\n',df['data_text'][satir][:df['data_text'][satir].find('Madde 1')],sep='')

Örnek belgede de gözüktüğü üzere **aradığımız değerler** dosyanın ilk parçasında mevcuttur.

**Kanun Hükmünde Kararname** kategorili toplam 91 belge mevcuttur. Belgelerden istenilen verilerin alınmasıyla ilgili takımımız 2 farklı yol bulmuştur.

1. TDDİ **"Question-Answering"** NLP görevi kullanılarak değerlerin eldesi.
2. Belgelerde bulunan ortak yazım kurallarını tespit ederek ve **Kural Bazlı Arama** yaparak değerlerin eldesi.

Kodun devamında **iki yöntem** de gösterilecek, **artı ve eksileri** hakkında tartışılacak ve yöntemlerin **başarı oranları** verilecektir.

# **1. Yöntem | TDDİ Kullanılarak Verilerin Eldesi**

TDDİ kullanmak için öncelikle **transformers** mödülü yüklenilecektir. Modül [Gereksinimler.txt](https://github.com/iflGARAJI10100/taMAM/blob/main/Gereksinimler.txt) de de belirtildiği gibi **4.20.1** sürümünde yüklenecektir.

In [ ]:
!pip install transformers==4.20.1

**"Question-Answering"** NLP görevini kullanırken ihtiyacımız olan **en işlevsel** modeli bulmak için [Hugging Face](https://huggingface.co/models?language=tr&pipeline_tag=question-answering&sort=downloads) sitesinden uygun modelleri araştırıyoruz.

Daha sonra sırasıyla bu modellere **örnek bir belge** verip o belgeden bize verdiği değerleri asıl değerler ile **karşılaştıracak**, **doğruluk oranı** en yüksek olan modeli de projemizde kullanacağız.



In [ ]:
# Modelleri kullanırken ihityacımız olan fonksiyonları çağrımakla başlıyoruz.
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# Test edeceğimiz modellerin listesi
modeller = ["lserinol/bert-turkish-question-answering","savasy/bert-base-turkish-squad","yunusemreemik/logo-qna-model","emre/distilbert-tr-q-a","oguzhanolm/loodos-bert-base-uncased-QA-fine-tuned"]

for x in modeller:
  # İçerik olarak önceki bulgularımıza dayanarak belgenin ilk kısmını seçiyoruz.
  icerik = df['data_text'][satir][:df['data_text'][satir].find('Madde 1')]
  
  # TDDİ yaparken kullanacağımız soruların listesini ve cevapları tanımlıyoruz.
  sorular = ["Resmi Gazete Tarihi nedir?","Resmi Gazete Sayısı nedir?","Karar Sayısı Nedir?","Kararnamenin Tarihi nedir?"]
  cevaplar = []

  # Model hazırlanır.
  veriBulucu = pipeline(task='question-answering', model=AutoModelForQuestionAnswering.from_pretrained(x), tokenizer=AutoTokenizer.from_pretrained(x))
  
  # Model adı verildikten sonra modelin sorulara verdiği cevaplar yazdırılır.
  print("Model:",x)
  for soru in sorular:
      sonuc = veriBulucu(question = soru, context = icerik)
      cevaplar.append(sonuc['answer'])
      print("'",soru,"'","sorusuna verilen cevap:",sonuc['answer'])
  print(35*'-')

# Karşılaştırma için asıl değerler yazdırılır.
print('Kategori: ',df['kategori'][satir],'\n','Resmi Gazete No: ',df['rega_no'][satir],'\n','Resmi Gazete Tarihi: ',df['rega_tarihi'][satir],'\n','Mevzuat No: ',df['mevzuat_no'][satir],'\n','Mevzuat Tarihi: ',df['mevzuat_tarihi'][satir],'\n',35*'-',sep='')

## ***ÇIKARIM:***

- **(ÖNEMLİ BULGU)** Basitçe Test ettiğimiz 5 modelden sadece 2' si istenilen performansı göstermiştir. Bu iki model de ileride daha detaylı test edilip yeniden en iyi performans göstereni seçilecektir.
  - lserinol/bert-turkish-question-answering
  - savasy/bert-base-turkish-squad 

Bu şekilde modellerin karşılaştırılması bittikten sonra **aranılan değerleri** bütün belgelerde aramaya veya iki modelimizn doğruluk oranlarını hesaplamaya geçebiliriz.


In [ ]:
# Modelleri kullanırken ihityacımız olan fonksiyonları çağrımakla başlıyoruz.
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# Performans ölçümünde kullanılacak değişkenlerin listelerin oluşturulması.
degerlerimiz = ["rega_tarihi","rega_no","mevzuat_no","mevzuat_tarihi"]
belgelerdekiToplamDegerSayisi = 91
belgelerdekiHataliDegerSayisi = [0] * len(degerlerimiz)

# Test edeceğimiz modellerin listesi
modeller = ["lserinol/bert-turkish-question-answering","savasy/bert-base-turkish-squad"]

# istenilen değerileri almamızı sağlayacak soruların listesi ve cevaplar listesi oluşturulur.
sorular = ["Resmi Gazete Tarihi nedir?","Resmi Gazete Sayısı nedir?","Karar Sayısı Nedir?","Kararnamenin Tarihi nedir?"]
cevaplar = []

for x in modeller:
  # Performans Ölçümleri Sıfırlanır
  degerlerimiz = ["rega_no","rega_tarihi","mevzuat_no","mevzuat_tarihi"]
  belgelerdekiToplamDegerSayisi = 91
  belgelerdekiHataliDegerSayisi = [0] * len(degerlerimiz)

  for satir in range(826,917):
    # İçerik olarak önceki bulgularımıza dayanarak belgenin ilk kısmını seçiyoruz.
    icerik = df['data_text'][satir][:df['data_text'][satir].find('Madde 1')]
  
    # Cevaplar listesi sıfırlanır.
    cevaplar = []

    # Model hazırlanır.
    veriBulucu = pipeline(task='question-answering', model=AutoModelForQuestionAnswering.from_pretrained(x), tokenizer=AutoTokenizer.from_pretrained(x))

    # Geçerli modelden değerler alınır.
    for soru in sorular:
      sonuc = veriBulucu(question = soru, context = icerik)
      cevaplar.append(sonuc['answer'])

    # Çekilen Değerlerin formatlanması.
    if cevaplar[0].find("\n") != -1:
      cevaplar[0] = cevaplar[0][:cevaplar[0].find("\n")]

    if cevaplar[0].find("-") != -1:
      regaTarihi = datetime.strptime(cevaplar[0],"%d-%m-%Y")
    elif cevaplar[0].find(".") != -1:
      regaTarihi = datetime.strptime(cevaplar[0],"%d.%m.%Y")
    elif cevaplar[0].find("/") != -1:
      regaTarihi = datetime.strptime(cevaplar[0],"%d/%m/%Y")

    regaNo = cevaplar[1]

    mevzuatNo = cevaplar[2][cevaplar[2].find('/')+1:]
    if cevaplar[3].find("\n") != -1:
      cevaplar[3] = cevaplar[3][:cevaplar[3].find("\n")]

    if cevaplar[3].find("-") != -1:
      mevzuatTarihi = datetime.strptime(cevaplar[3],"%d-%m-%Y")
    elif cevaplar[3].find(".") != -1:
      mevzuatTarihi = datetime.strptime(cevaplar[3],"%d.%m.%Y")
    elif cevaplar[3].find("/") != -1:
      mevzuatTarihi = datetime.strptime(cevaplar[3],"%d/%m/%Y")

    # Asıl Değerlerin formatlanması.
    dfregaTarihi = datetime.strptime(df["rega_tarihi"][satir],"%Y-%m-%d")
    dfregaNo = df["rega_no"][satir]
    dfmevzuatTarihi = datetime.strptime(df["mevzuat_tarihi"][satir],"%Y-%m-%d")
    dfmevzuatNo = df["mevzuat_no"][satir]

    # Değerlerin Asılları ile Karşılaştırılması
    if regaTarihi != dfregaTarihi:
      belgelerdekiHataliDegerSayisi[0]+=1

    if regaNo != dfregaNo:
      belgelerdekiHataliDegerSayisi[1]+=1
      
    if mevzuatNo != dfmevzuatNo:
      belgelerdekiHataliDegerSayisi[2]+=1

    if mevzuatTarihi != dfmevzuatTarihi:
      belgelerdekiHataliDegerSayisi[3]+=1

  # Ölçüm bittiğinde Model ve Performansı Yazdırılır (İşlem Toplam 10dk civarı sürer)    
  print("\nModel: ",x) 
  for i in range(4):
    print(degerlerimiz[i],"|",belgelerdekiHataliDegerSayisi[i],"/",belgelerdekiToplamDegerSayisi,"|",round((belgelerdekiToplamDegerSayisi-belgelerdekiHataliDegerSayisi[i])/belgelerdekiToplamDegerSayisi,3))


## ***ÇIKARIM:***

- **(ÖNEMLİ BULGU)** 2 Modelden Daha Yüksek Performans gösteren Model **savasy/bert-base-turkish-squad** olmuştur. Bu yüzden bundan sonraki kodlarda model olarak o kullanılacaktır.

- **(ÖNEMLİ BULGU)** **savasy/bert-base-turkish-squad** modeli **"mevzuat_tarihi"**
değeri hariç diğer tüm değerlerde **Hatasız** çalışmaktadır.

- **savasy/bert-base-turkish-squad** modeli **"mevzuat_tarihi"**
değerinde  **Hatalı** çalışmaktadır.

Görüldüğü üzere modelin performansını **Hatasız** hale getimek için **"mevzuat_tarihi"** değerinde çalışmalar yapılmalıdır.

## Modeller'in Performans Tabloları
**1. lserinol/bert-turkish-question-answering**

|lserinol/bert-turkish-question-answering|Toplam|Doğru|Yanlış|Doğruluk Oranı|
|---|---|---|---|---|
|rega_tarihi|91|87|4|0.956|
|rega_no|91|91|0|1.0|
|mevzuat_tarihi|91|51|40|0.56|
|mevzuat_no|91|91|0|1.0|

**2. savasy/bert-base-turkish-squad**

|savasy/bert-base-turkish-squad|Toplam|Doğru|Yanlış|Doğruluk Oranı|
|---|---|---|---|---|
|rega_tarihi|91|91|0|1.0|
|rega_no|91|91|0|1.0|
|mevzuat_tarihi|91|70|21|0.769|
|mevzuat_no|91|91|0|1.0|

Şimdi **savasy/bert-base-turkish-squad** modelinin neden **mevzuat_tarihi** değerinde bu kadar hata verdiğine bakacağız. Hipotezlerimizden biri modelin onu o satırın **rega_tarihi** tahminiyle karıştırmasıdır. Şimdi bunu ölçmeyi deneyeceğiz.

In [ ]:
# Modelleri kullanırken ihityacımız olan fonksiyonları çağrımakla başlıyoruz.
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# Performans ölçümünde kullanılacak değişkenlerin listelerin oluşturulması.
degerlerimiz = ["rega_tarihi","rega_no","mevzuat_no","mevzuat_tarihi"]
belgelerdekiToplamDegerSayisi = 91
belgelerdekiHataliDegerSayisi = [0] * len(degerlerimiz)

# Test edeceğimiz modellerin listesi
modeller = ["savasy/bert-base-turkish-squad"]

# istenilen değerileri almamızı sağlayacak soruların listesi ve cevaplar listesi oluşturulur.
sorular = ["Resmi Gazete Tarihi nedir?","Resmi Gazete Sayısı nedir?","Karar Sayısı Nedir?","Kararnamenin Tarihi nedir?"]
cevaplar = []

for x in modeller:
  # Performans Ölçümleri Sıfırlanır
  degerlerimiz = ["rega_no","rega_tarihi","mevzuat_no","mevzuat_tarihi"]
  belgelerdekiToplamDegerSayisi = 91
  belgelerdekiHataliDegerSayisi = [0] * len(degerlerimiz)
  ayniTarih = 0

  for satir in range(826,917):
    # İçerik olarak önceki bulgularımıza dayanarak belgenin ilk kısmını seçiyoruz.
    icerik = df['data_text'][satir][:df['data_text'][satir].find('Madde 1')]
  
    # Cevaplar listesi sıfırlanır.
    cevaplar = []

    # Model hazırlanır.
    veriBulucu = pipeline(task='question-answering', model=AutoModelForQuestionAnswering.from_pretrained(x), tokenizer=AutoTokenizer.from_pretrained(x))

    # Geçerli modelden değerler alınır.
    for soru in sorular:
      sonuc = veriBulucu(question = soru, context = icerik)
      cevaplar.append(sonuc['answer'])

    # Çekilen Değerlerin formatlanması.
    if cevaplar[0].find("\n") != -1:
      cevaplar[0] = cevaplar[0][:cevaplar[0].find("\n")]

    if cevaplar[0].find("-") != -1:
      regaTarihi = datetime.strptime(cevaplar[0],"%d-%m-%Y")
    elif cevaplar[0].find(".") != -1:
      regaTarihi = datetime.strptime(cevaplar[0],"%d.%m.%Y")
    elif cevaplar[0].find("/") != -1:
      regaTarihi = datetime.strptime(cevaplar[0],"%d/%m/%Y")

    regaNo = cevaplar[1]

    mevzuatNo = cevaplar[2][cevaplar[2].find('/')+1:]

    if cevaplar[3].find("\n") != -1:
      cevaplar[3] = cevaplar[3][:cevaplar[3].find("\n")]

    if cevaplar[3].find("-") != -1:
      mevzuatTarihi = datetime.strptime(cevaplar[3],"%d-%m-%Y")
    elif cevaplar[3].find(".") != -1:
      mevzuatTarihi = datetime.strptime(cevaplar[3],"%d.%m.%Y")
    elif cevaplar[3].find("/") != -1:
      mevzuatTarihi = datetime.strptime(cevaplar[3],"%d/%m/%Y")

    # Asıl Değerlerin formatlanması.
    dfregaTarihi = datetime.strptime(df["rega_tarihi"][satir],"%Y-%m-%d")
    dfregaNo = df["rega_no"][satir]
    dfmevzuatTarihi = datetime.strptime(df["mevzuat_tarihi"][satir],"%Y-%m-%d")
    dfmevzuatNo = df["mevzuat_no"][satir]

    # Değerlerin Asılları ile Karşılaştırılması
    if regaTarihi != dfregaTarihi:
      belgelerdekiHataliDegerSayisi[0]+=1

    if regaNo != dfregaNo:
      belgelerdekiHataliDegerSayisi[1]+=1

    if mevzuatNo != dfmevzuatNo:
      belgelerdekiHataliDegerSayisi[2]+=1

    if mevzuatTarihi != dfmevzuatTarihi:
      belgelerdekiHataliDegerSayisi[3]+=1
      # Hipotezimizin denenmesi
      if mevzuatTarihi == regaTarihi:
        ayniTarih +=1

    

  # Ölçüm bittiğinde Model ve Performansı Yazdırılır (Artık işlem süresi 4dk 40 sn'ye düşmüştür)    
  print("Model: ",x) 
  for i in range(4):
    print(degerlerimiz[i],"|",belgelerdekiHataliDegerSayisi[i],"/",belgelerdekiToplamDegerSayisi,"|",round((belgelerdekiToplamDegerSayisi-belgelerdekiHataliDegerSayisi[i])/belgelerdekiToplamDegerSayisi,3))
  print("Aynı Tarih Hata:",ayniTarih,"/",belgelerdekiHataliDegerSayisi[3])

## ***ÇIKARIM:***

- **(ÖNEMLİ BULGU)** Modelimiz 91 Belgeden **21**'inde  **"mevzuat_tarihi"**ni **"rega_tarihi"** değeriyle karıştırmaktadır. Bu sorun çözülürse modelin performansı istenilen seviyeye ulaştırılabilir.

## **YÖNTEMİN ARTI VE EKSİLERİ**
### Artıları:
- TDDİ kullanılarak modülün daha önce karşılaşmadığı metinleri doğru işleme olasılığı arttırılıyor.

### Eksileri:
- Seçilen bir modelin başka belge tiplerinde aynı performansı sergileyeceği belirsiz.
- Performans'ın arttırılma çalışmaları emek istiyor.
- Kodların çalışma süreleri diğer alternatiflere göre fazla uzun sürüyor.




# **2. Yöntem | Kural Bazlı Sistem ile Verilerin Eldesi**

Bu Yöntemde ise Belgeler arası **ortak yazım şekillerine** dikkat edilerek veriler elde edilir.

In [ ]:
#Performans ölçümü için gerekli olan değişkenlerin oluşturulması
degerlerimiz = ["rega_tarihi","rega_no","mevzuat_no","mevzuat_tarihi"]
belgelerdekiToplamDegerSayisi = 91
belgelerdekiHataliDegerSayisi = [0] * len(degerlerimiz)

# Sadece KHK kategorisindeki belgelerde işlem yapıyoruz.
for i in range(826,917):
  # VERİ TEMİZLEME İŞLEMLERİ
  metin = df['data_text'][i][:df['data_text'][i].find('Madde 1')].lower()
  metin = metin.replace('  ','')


  # 'rega_tarihi' değerini bulan kod
  regaTarihiIndex = metin.find('resmî gazete tarihi:')
  regaTarihiIndex = regaTarihiIndex + len('resmî gazete tarihi: ')
  regaTarihi= metin[regaTarihiIndex:regaTarihiIndex+10]

  
  # 'rega_no' değerini bulan kod
  regaNoIndex = metin.find('resmî gazete sayısı: ')
  regaNoIndex = regaNoIndex + len('resmî gazete sayısı: ')
  regaNo = ''
  rakambuldu = 0
  while metin[regaNoIndex].isdigit() or rakambuldu==0:
    if metin[regaNoIndex] == ' ' or metin[regaNoIndex] == '\n':
      rakambuldu = 0
    elif metin[regaNoIndex].isdigit():
      rakambuldu = 1
      regaNo = regaNo + metin[regaNoIndex]
    regaNoIndex = regaNoIndex + 1
  regaNo = int(regaNo)

  
  # 'mevzuat_no' değerini bulan kod
  mevzuatNoIndex = metin.find('karar sayısı : ')
  mevzuatNoIndex = mevzuatNoIndex + len('karar sayısı : ') + 4
  mevzuatNo = ''
  rakambuldu = 0
  while metin[mevzuatNoIndex].isdigit() or rakambuldu==0:
    if metin[mevzuatNoIndex] == ' ' or metin[mevzuatNoIndex] == '\n':
      rakambuldu = 0
    elif metin[mevzuatNoIndex].isdigit():
      rakambuldu = 1
      mevzuatNo = mevzuatNo + metin[mevzuatNoIndex]
    mevzuatNoIndex = mevzuatNoIndex + 1
  mevzuatNo = int(mevzuatNo)
  

  # 'mevzuat_tarihi' değerini bulan kod
  mevzuatTarihiIndex = metin.find('kararnamenin tarihi : ')
  mevzuatTarihiIndex = mevzuatTarihiIndex + len('kararnamenin tarihi : ')
  mevzuatTarihi = metin[mevzuatTarihiIndex:mevzuatTarihiIndex+10]

  mevzuatTarihi = mevzuatTarihi.replace('\n', '')
  mevzuatTarihi = mevzuatTarihi.replace(' ', '')
  

  # Asıl değerlerin dosyadan çekilmesi
  dfRegaTarihi = df.loc[df['kategori'] == 'Kanun Hükmünde Kararname','rega_tarihi'][i]
  dfRegaNo = int(df.loc[df['kategori'] == 'Kanun Hükmünde Kararname','rega_no'][i])
  dfMevzuatNo = int(df.loc[df['kategori'] =='Kanun Hükmünde Kararname','mevzuat_no'][i])
  dfMevzuatTarihi = df.loc[df['kategori'] == 'Kanun Hükmünde Kararname','mevzuat_tarihi'][i]

  # Asıl değerlerin formatlanması
  dfRegaTarihi = datetime.strptime(dfRegaTarihi, '%Y-%m-%d')
  dfMevzuatTarihi = datetime.strptime(dfMevzuatTarihi, '%Y-%m-%d')

  # Çekilen Değerlerin formatlanması
  if regaTarihi.find("\n") != -1:
    regaTarihi = regaTarihi[:regaTarihi.find("\n")]

    if regaTarihi.find("-") != -1:
      regaTarihi = datetime.strptime(regaTarihi,"%d-%m-%Y")
    elif regaTarihi.find(".") != -1:
      regaTarihi = datetime.strptime(regaTarihi,"%d.%m.%Y")
    elif regaTarihi.find("/") != -1:
      regaTarihi = datetime.strptime(regaTarihi,"%d/%m/%Y")

  if mevzuatTarihi.find("\n") != -1:
    mevzuatTarihi = mevzuatTarihi[:mevzuatTarihi.find("\n")]

    if mevzuatTarihi.find("-") != -1:
      mevzuatTarihi = datetime.strptime(mevzuatTarihi,"%d-%m-%Y")
    elif mevzuatTarihi.find(".") != -1:
      mevzuatTarihi = datetime.strptime(mevzuatTarihi,"%d.%m.%Y")
    elif mevzuatTarihi.find("/") != -1:
      mevzuatTarihi = datetime.strptime(mevzuatTarihi,"%d/%m/%Y")

  # Çekilen Değerlerin Asıllarıyla Karşılaştırılması

  # Değerlerin Asılları ile Karşılaştırılması
    if regaTarihi != dfregaTarihi:
      belgelerdekiHataliDegerSayisi[0]+=1

    if regaNo != dfregaNo:
      belgelerdekiHataliDegerSayisi[1]+=1

    if mevzuatNo != dfmevzuatNo:
      belgelerdekiHataliDegerSayisi[2]+=1

    if mevzuatTarihi != dfmevzuatTarihi:
      belgelerdekiHataliDegerSayisi[3]+=1

# Kodun Performansının Yazdırılması
for i in range(4):
    print(degerlerimiz[i],"|",belgelerdekiHataliDegerSayisi[i],"/",belgelerdekiToplamDegerSayisi,"|",round((belgelerdekiToplamDegerSayisi-belgelerdekiHataliDegerSayisi[i])/belgelerdekiToplamDegerSayisi,3))
  

## ***ÇIKARIM:***

- **(ÖNEMLİ BULGU)** Bu yöntemle modülümüz istenilen performans düzeyine ulaşmıştır. Bütün istenilen değerleri **Hatasız** şekilde bulabilmiştir.

## **YÖNTEMİN ARTI VE EKSİLERİ**
### Artıları:
- Kural Bazlı Arama'sı diğer yöntemlere hız bakımından üstün.
- Kural Bazlı Arama'sı 1.0 Doğruluk Performansıyla çalışıyor.

### Eksileri:
- Kural Bazlı Arama'nın diğer belge tiplerinde nasıl çalışacağı bilinmiyor.
- Kural Bazlı Arama'nın yeni yazım kurallarına göre yazılmış metinlerde Doğruluk oranı 0.0'a kadar düşebilir, riskli.

### **Yöntemin Performans Tablosu**

|Kural Bazlı Arama Yöntemi|Toplam|Doğru|Yanlış|Doğruluk Oranı|
|---|---|---|---|---|
|rega_tarihi|91|91|0|1.0|
|rega_no|91|91|0|1.0|
|mevzuat_tarihi|91|91|0|1.0|
|mevzuat_no|91|91|0|1.0|